### Архитектура (в кратком виде):

```python
+-----------------+
| CCXT API        |
| (fetch OHLCV)   |
+--------+--------+
         |
         v
+--------+--------+
| Предобработка   |  <-- TA-Lib, TA, Pandas
+--------+--------+
         |
         v
+--------+--------+
| Технические     |  <-- RSI, MACD, BB, etc
| Индикаторы      |
+--------+--------+
         |
         v
+--------+--------+
| ML-модель       |  <-- Scikit-learn или XGBoost
+--------+--------+
         |
         v
+--------+--------+
| Веб-хуки        |  <-- Сигналы в Bybit
```


#### Про библиотеки

**ccxt (CryptoCurrency eXchange Trading Library)** — это библиотека для работы с различными криптовалютными биржами. Она предоставляет унифицированный интерфейс для доступа к API более чем 100 криптобирж, что позволяет разработчикам легко интегрировать функции торговли и получения данных о рынках.

**Основные функции:**
- Получение рыночных данных: Позволяет загружать данные о ценах, объемах и других рыночных показателях.
- Торговля: Поддерживает выполнение ордеров, получение информации о счетах и управление активами.
- Поддержка различных бирж: Обеспечивает совместимость с множеством популярных бирж, таких как Binance, Coinbase, Kraken и другие.
- Применение: Используется для создания торговых ботов, анализа рынка и автоматизации торговли.
-  ИНФОРМЦИЯ ПО БИБЛИОТЕКИ https://github.com/ccxt/ccxt/wiki/Manual

`pip install ta`

**ta (Technical Analysis Library)** — это библиотека для технического анализа, которая предоставляет удобный интерфейс для работы с индикаторами и сигналами. Она интегрируется с Pandas и позволяет легко добавлять индикаторы в DataFrame.
Основные функции:
- Индикаторы: Включает множество индикаторов, таких как SMA, EMA, RSI, MACD и другие.
- Простота использования: Позволяет легко добавлять индикаторы в DataFrame, что делает анализ данных более удобным.
- Гибкость: Поддерживает различные параметры для настройки индикаторов.
- Применение: Используется для анализа финансовых данных, создания торговых стратегий и визуализации результатов.

`pip install ta`

**talib (Technical Analysis Library)** — это библиотека для технического анализа финансовых данных. Она предоставляет множество индикаторов и функций для анализа временных рядов, таких как скользящие средние, RSI, MACD и многие другие.
**Основные функции:**
- Индикаторы: Включает более 150 различных индикаторов, которые можно использовать для анализа рыночных данных.
- Поддержка различных форматов данных: Может работать с массивами данных, полученными из различных источников, таких как Pandas DataFrame.
-Оптимизация производительности: Написана на C, что обеспечивает высокую производительность при вычислениях.
- Применение: Используется для анализа исторических данных, создания торговых стратегий и принятия решений на основе технического анализа

Как её себе поставить:
1. *(ТЕРМИНАЛ)* узнать версию питона: pip --version
1. Перейти сюды: https://github.com/cgohlke/talib-build/releases
1. Выбрать нужную версию и скачать.
1. *(ТЕРМИНАЛ)* Как выбрать? На основе версии питона и системы своей 32 или 64 бита: `python -c "import platform; print(platform.architecture())"`
1. *(ТЕРМИНАЛ)* Пример, у меня Python 3.10 и 64-битная система, и я скачал `ta_lib-0.6.3-cp310-cp310-win_amd64.whl`
1. *(ТЕРМИНАЛ)* Переходим в папку с загрузкой: `cd C:\Users\xndrf\Downloads `- это моё. 
1. *(ТЕРМИНАЛ)* Под свою версию питона ставим ``py -3.10 -m pip install ta_lib-0.6.3-cp310-cp310-win_amd64.whl``
1. *(ТЕРМИНАЛ)* Проверим установку: `py -3.10 -c "import talib; print(talib.get_functions())"`
1. И теперь пойдет у нас импорт `import talib`

In [1]:
# Импорт библиотек
import ccxt
import pandas as pd
import time
import talib
import ta

### 1. Получение исторических данных с CCXT

In [2]:
def fetch_ohlcv_data(exchange_name, symbol, timeframe='1h', limit=1000):
    """Функция предназначена для получения исторических данных о ценах
    на криптовалюту в формате OHLCV с указанной криптобиржи

    Args:
        exchange_name (str): название криптобиржи (например, 'binance')
        symbol (str): Торговая пара (например, 'BTC/USDT').
        timeframe (str): Период времени для каждой свечи. Стандартно - 1h.
        limit (int): Максимальное количество свечей. Стандарт 1000.

    Returns:
        _type_: Датасет с собранными данными
    """    
    exchange_class = getattr(ccxt, exchange_name)()  #создаем экземпляр биржи
    exchange_class.load_markets() # загружаем рынки
    
    # Проверяем доступность торговой пары, есть или нет
    if symbol not in exchange_class.symbols:
        raise Exception(f"{symbol} not available on {exchange_name}")
    # Получаем OHLCV
    ohlcv = exchange_class.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    # Всё это в датафрейм кидаем
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    # И временные метки лепим
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df


df = fetch_ohlcv_data('bitfinex', 'BTC/USDT', '1d', 500)
df

,timestamp,open,high,low,close,volume
0,2019-03-11,3870.1,3907.7,3850.0,3866.1,8.692549
1,2019-03-12,3866.1,3900.7,3821.9,3881.3,47.018426
2,2019-03-13,3885.9,3889.1,3846.1,3863.1,9.193549
3,2019-03-14,3873.6,3897.0,3800.9,3870.8,33.271493
4,2019-03-15,3892.0,3932.9,3886.6,3927.9,8.248969
...,...,...,...,...,...,...
495,2020-07-18,9151.7,9198.5,9124.6,9168.2,298.763918
496,2020-07-19,9169.0,9228.0,9105.7,9210.9,164.052004
497,2020-07-20,9214.9,9215.2,9132.2,9159.7,140.380323
498,2020-07-21,9157.7,9425.7,9155.8,9382.5,375.508933


<div class='alert alert-info'>

- <span style="color:#000000">**Криптобиржи**</span> - <span style="color:#02020A">['binance', 'coinbase', 'kraken', 'bitfinex, 'huobi']</span>.

    Список криптобирж - `print(ccxt.exchanges)` или https://github.com/ccxt/ccxt/wiki/Exchange-Markets

- <span style="color:#000000">**Торговые пары**</span> - <span style="color:#02020A">['BTC/USDT', 'ETH/USDT', 'XRP/BTC', 'LTC/USD, 'ADA/USDT']</span>.
- <span style="color:#000000">**Временные интервалы**</span> - <span style="color:#02020A">['1m', '5m', '15m', '30m, '1h', '4h', '1d']</span>.
- <span style="color:#000000">**Количество свечей**</span> - <span style="color:#02020A">['100', '500', '1000', 'bitfinex, 'huobi']</span>.

### 2. Расчёт технических индикаторов (с помощью библиотеки ta)

In [3]:
def add_technical_indicators(df):
    """Функция для добавления различных технических индикаторов в DF/

    Args:
        df: Датафрейм после сбора

    Returns:
        df : Датафрейм с индикатарами
    """    
    # Cохраним исходные данные нетронутыми.
    df = df.copy()

    # Трендовые индикаторы
    # Вычисляет 20-дневную простую скользящую среднюю на основе цен закрытия и по закрытию
    df['sma_20'] = ta.trend.SMAIndicator(close=df['close'], window=20).sma_indicator()
    df['ema_20'] = ta.trend.EMAIndicator(close=df['close'], window=20).ema_indicator()
    # Индикатор Ишимоку предоставляет информацию о поддержке и сопротивлении, а также о направлении тренда.
    ichimoku = ta.trend.IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52)
    df['ichimoku_a'] = ichimoku.ichimoku_a()
    df['ichimoku_b'] = ichimoku.ichimoku_b()

    # Осцилляторы
    # Индикатор относительной силы, который измеряет скорость и изменение ценовых движений.
    # Значения выше 70 указывают на перекупленность, а ниже 30 — на перепроданность.
    df['rsi'] = ta.momentum.RSIIndicator(close=df['close'], window=14).rsi()
    # Индикатор, который показывает разницу между 12-дневной и 26-дневной экспоненциальными скользящими средними.
    df['macd'] = ta.trend.MACD(close=df['close']).macd_diff()
    # Индикатор, который сравнивает текущую цену закрытия с диапазоном цен за определенный период.
    df['stochastic'] = ta.momentum.StochasticOscillator(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        window=14,
        smooth_window=3
    ).stoch()

    # # Объёмные индикаторы
    # Индикатор, который использует объем для подтверждения тренда. Если цена растет, объем добавляется, если падает — вычитается.
    df['obv'] = ta.volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()
    
    
    # !ХЗ чо делать, ошибку выдает.. надо ли этот индикатор
    # Индикатор, который показывает среднюю цену актива с учетом объема торгов.
    # df['vwap'] = ta.volume.VolumeWeightedAveragePrice(
    #     high=df['high'],
    #     low=df['low'],
    #     close=df['close'],
    #     volume=df['volume'],
    #     window=14
    # ).vwap()

    # Волатильность
    # Полосы Боллинджера состоят из средней скользящей и двух стандартных отклонений, которые помогают определить уровни поддержки и сопротивления.
    bollinger = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_bbm'] = bollinger.bollinger_mavg()
    df['bb_bbh'] = bollinger.bollinger_hband()
    df['bb_bbl'] = bollinger.bollinger_lband()
    
    # Индикатор, который измеряет волатильность актива, показывая средний диапазон цен за определенный период.
    df['atr'] = ta.volatility.AverageTrueRange(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        window=14
    ).average_true_range()
    
    return df

# Применение к DataFrame df
df = add_technical_indicators(df)
df

,timestamp,open,high,low,close,volume,sma_20,ema_20,ichimoku_a,ichimoku_b,rsi,macd,stochastic,obv,bb_bbm,bb_bbh,bb_bbl,atr
0,2019-03-11,3870.1,3907.7,3850.0,3866.1,8.692549,NaN,NaN,NaN,3878.85,NaN,NaN,NaN,8.692549,NaN,NaN,NaN,0.000000
1,2019-03-12,3866.1,3900.7,3821.9,3881.3,47.018426,NaN,NaN,NaN,3864.80,NaN,NaN,NaN,55.710975,NaN,NaN,NaN,0.000000
2,2019-03-13,3885.9,3889.1,3846.1,3863.1,9.193549,NaN,NaN,NaN,3864.80,NaN,NaN,NaN,46.517426,NaN,NaN,NaN,0.000000
3,2019-03-14,3873.6,3897.0,3800.9,3870.8,33.271493,NaN,NaN,NaN,3854.30,NaN,NaN,NaN,79.788919,NaN,NaN,NaN,0.000000
4,2019-03-15,3892.0,3932.9,3886.6,3927.9,8.248969,NaN,NaN,NaN,3866.90,NaN,NaN,NaN,88.037889,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-07-18,9151.7,9198.5,9124.6,9168.2,298.763918,9207.685,9229.942416,9235.750,9605.65,46.789239,-2.828665,45.891784,29131.475448,9207.685,9389.401199,9025.968801,200.407626
496,2020-07-19,9169.0,9228.0,9105.7,9210.9,164.052004,9208.870,9228.128852,9225.125,9605.65,48.634156,-0.113641,39.610544,29295.527452,9208.870,9390.345344,9027.394656,194.828510
497,2020-07-20,9214.9,9215.2,9132.2,9159.7,140.380323,9210.040,9221.611819,9173.850,9605.65,46.550030,-1.387874,27.451912,29155.147129,9210.040,9389.923522,9030.156478,186.840759
498,2020-07-21,9157.7,9425.7,9155.8,9382.5,375.508933,9217.670,9236.934503,9194.825,9605.65,55.488870,12.393504,80.360959,29530.656062,9217.670,9412.592601,9022.747399,192.773562


<div class='alert alert-warning'>

- <span style="color:#000000">**Криптобиржи**</span> - <span style="color:#02020A">['binance', 'coinbase', 'kraken', 'bitfinex, 'huobi']</span>.

Также уществует множество других технических индикаторов, которые можно добавить для более глубокого анализа рынка. 
Пока правда не ясно что с ними со всеми делать, но вот на всякий списочек:
- <span style="color:#000000">**Трендовые индикаторы**</span>.
1. **Hull Moving Average (HMA)**: Быстрая и гладкая скользящая средняя, которая уменьшает запаздывание.
1. **Arnaud Legoux Moving Average (ALMA)**: Скользящая средняя, которая учитывает весовые коэффициенты для более точного отображения тренда.
1. **Supertrend**: Индикатор, который помогает определить направление тренда и уровни поддержки/сопротивления.
- <span style="color:#000000">**Осцилляторы**</span>.
1. **Stochastic RSI (StochRSI)**: Осциллятор, который измеряет уровень перекупленности или перепроданности на основе RSI.
1. **Commodity Channel Index (CCI)**: Индикатор, который измеряет отклонение текущей цены от средней цены за определенный период.
1. **KDJ**: Осциллятор, который сочетает в себе элементы стохастического осциллятора и RSI.
- <span style="color:#000000">**Объёмные индикаторы**</span>.
1. **Chaikin Money Flow (CMF)**: Индикатор, который измеряет объемный поток денег, учитывая как цену, так и объем.
1. **Accumulation/Distribution Line (A/D)**: Индикатор, который показывает, как объем соотносится с ценой, чтобы определить, идет ли накопление или распределение актива.**</span>.
1. **Klinger Volume Oscillator (KVO)**: Индикатор, который использует объем для определения тренда.
- <span style="color:#000000">**Волатильность**</span>.
1. **Average True Range (ATR)**: Измеряет волатильность актива, показывая средний диапазон цен за определенный период.
1. **Bollinger Bands**: Полосы, которые показывают уровни поддержки и сопротивления на основе стандартного отклонения от скользящей средней.
1. **Keltner Channel**: Канал, который использует ATR для определения уровней поддержки и сопротивления.
- <span style="color:#000000">**Другие индикаторы**</span>.
1. **Elder Ray Index**: Индикатор, который показывает силу быков и медведей на рынке.
1. **Mass Index**: Индикатор, который помогает определить возможные развороты тренда.
1. **Donchian Channel**: Индикатор, который показывает максимумы и минимумы за определенный период, что помогает определить уровни поддержки и сопротивления.


Если необходимо доп, то вот функция:


```python
def add_technical_indicators(df):
    # Убедимся, что индикаторы не дублируются
    df = df.copy()

    # Трендовые индикаторы
    df['hma'] = ta.trend.HullMovingAverage(close=df['close'], window=14).hull_moving_average()
    df['alma'] = ta.trend.ALMAClose(close=df['close'], window=14).alma()

    # Осцилляторы
    df['stochrsi'] = ta.momentum.StochasticOscillator(
        close=df['close'],
        window=14,
        smooth_window=3
    ).stochrsi()

    df['cci'] = ta.trend.CCIIndicator(high=df['high'], low=df['low'], close=df['close'], window=20).cci()

    # Объёмные индикаторы
    df['cmf'] = ta.volume.ChaikinMoneyFlowIndicator(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        volume=df['volume'],
        window=20
    ).chaikin_money_flow()

    # Волатильность
    df['atr'] = ta.volatility.AverageTrueRange(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        window=14
    ).average_true_range()

    return df
```

### 3. Распознавание свечных паттернов (молот, звезда и пр.)

In [4]:
def add_candlestick_patterns(df):
    open = df['open'].values
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values

    df['hammer'] = talib.CDLHAMMER(open, high, low, close)
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(open, high, low, close)
    df['engulfing'] = talib.CDLENGULFING(open, high, low, close)
    
    return df

# Assuming df is your DataFrame with OHLC data
df = add_candlestick_patterns(df)

In [5]:
df

,timestamp,open,high,low,close,volume,sma_20,ema_20,ichimoku_a,ichimoku_b,...,macd,stochastic,obv,bb_bbm,bb_bbh,bb_bbl,atr,hammer,shooting_star,engulfing
0,2019-03-11,3870.1,3907.7,3850.0,3866.1,8.692549,NaN,NaN,NaN,3878.85,...,NaN,NaN,8.692549,NaN,NaN,NaN,0.000000,0,0,0
1,2019-03-12,3866.1,3900.7,3821.9,3881.3,47.018426,NaN,NaN,NaN,3864.80,...,NaN,NaN,55.710975,NaN,NaN,NaN,0.000000,0,0,0
2,2019-03-13,3885.9,3889.1,3846.1,3863.1,9.193549,NaN,NaN,NaN,3864.80,...,NaN,NaN,46.517426,NaN,NaN,NaN,0.000000,0,0,-100
3,2019-03-14,3873.6,3897.0,3800.9,3870.8,33.271493,NaN,NaN,NaN,3854.30,...,NaN,NaN,79.788919,NaN,NaN,NaN,0.000000,0,0,0
4,2019-03-15,3892.0,3932.9,3886.6,3927.9,8.248969,NaN,NaN,NaN,3866.90,...,NaN,NaN,88.037889,NaN,NaN,NaN,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-07-18,9151.7,9198.5,9124.6,9168.2,298.763918,9207.685,9229.942416,9235.750,9605.65,...,-2.828665,45.891784,29131.475448,9207.685,9389.401199,9025.968801,200.407626,0,0,0
496,2020-07-19,9169.0,9228.0,9105.7,9210.9,164.052004,9208.870,9228.128852,9225.125,9605.65,...,-0.113641,39.610544,29295.527452,9208.870,9390.345344,9027.394656,194.828510,0,0,0
497,2020-07-20,9214.9,9215.2,9132.2,9159.7,140.380323,9210.040,9221.611819,9173.850,9605.65,...,-1.387874,27.451912,29155.147129,9210.040,9389.923522,9030.156478,186.840759,0,0,-100
498,2020-07-21,9157.7,9425.7,9155.8,9382.5,375.508933,9217.670,9236.934503,9194.825,9605.65,...,12.393504,80.360959,29530.656062,9217.670,9412.592601,9022.747399,192.773562,0,0,100


### 4. Подготовка данных для ML

In [6]:
def prepare_ml_dataset(df):
    df = df.dropna()
    features = [
        'sma_20', 'ema_20', 'ichimoku_a', 'ichimoku_b',
        'rsi', 'macd', 'stochastic', 'obv', #'vwap',
        'bb_bbm', 'bb_bbh', 'bb_bbl', 'atr',
        'hammer', 'shooting_star', 'engulfing'
    ]
    target = df['close'].shift(-1) > df['close']  # бинарный таргет: 1 если цена вырастет
    return df[features], target.astype(int)

X, y = prepare_ml_dataset(df)

### 5. Обучение простой модели ML (например, RandomForest)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.98      0.62        41
           1       0.80      0.08      0.14        53

    accuracy                           0.47        94
   macro avg       0.62      0.53      0.38        94
weighted avg       0.65      0.47      0.35        94



### 6. Отправка сигналов на Bybit через Webhook


```python
import requests
from datetime import datetime

def send_webhook(symbol, side, qty, price, strategy_id="test123"):
    url = 'https://api.bybit.com/custom_webhook_endpoint'
    data = {
        "symbol": symbol,
        "side": side,
        "qty": str(qty),
        "price": str(price),
        "trigger_time": datetime.utcnow().isoformat(),
        "max_lag": "20",
        "strategy_id": strategy_id
    }
    response = requests.post(url, json=data)
    print(response.status_code, response.text)

# Пример сигнала
send_webhook("BTC/USDT", "buy", 1, df['close'].iloc[-1])
```


***Не ясно как это все дело будет работа***